# Demonstration of Validation Warnings

This notebook showcases how validation warnings are triggered in the SDK. Some cells intentionally contain errors to illustrate the validation process. You do not need to run all cells sequentially.

In [1]:
# import client and authenticate
from uncertainty_engine import Client
client = Client()
client.authenticate()

## Client required to perform validation

### Node Instantiation Without Client
Validation requires the `client` to be passed to various components, such as nodes and workflows. Instantiating a node without the required `client` parameter triggers validation warnings. Similar issues can occur if the `client` is omitted elsewhere.

In [2]:
# instantiate a node without the client parameter
from uncertainty_engine.nodes.base import Node

number_node = Node(
    node_name="Number",
    value=42,
    label="number_node_without_client",
)

### Node Instantiation With Client
Instantiate a node with the `client` parameter to get node parameter validation whilst building a workflow.

In [3]:
# instantiate a node with the client parameter

number_node = Node(
    node_name="Number",
    value=42,
    label="number_node_with_client",
    client=client,
)

## Node instantiation

Compare using a custom node class versus the base `Node` class. Validation warnings may occur depending on the class used. Using the base `Node` class to instantiate a node type that hasn't been deployed triggers a validation warning.

In [4]:
# using an existing custom class
from uncertainty_engine.nodes.basic import Add

first_add = Add(
    lhs=5,
    rhs=10,
    label="first_add_node",
    client=client,
)

In [5]:
# using the base Node class
from uncertainty_engine.nodes.base import Node

second_add = Node(
    node_name="Add",
    lhs=5,
    label="second_add_node",
    client=client,
)

Using the base `Node` class to instantiate a node type that does not exist will raise a "404: not found" error.

In [6]:
# instantiating a node that is not deployed
my_node = Node(
    node_name="My Node",
    my_param=1,
    label="my_node",
    client=client,
)

HTTPError: 404 Client Error: Not Found for url: https://de1v75vvk6.execute-api.eu-west-2.amazonaws.com/nodes/My%20Node

## Handle creation

Handles should reference existing node outputs. Using `make_handle` on a node instantiated with the client to show a warning if a handle name is incorrect and list available outputs for that node.

In [7]:
name_handle = second_add.make_handle("sum")

In [8]:
ans_handle = second_add.make_handle("ans")

## Adding a node to the graph

Adding nodes to a graph triggers validation. If a node with a duplicate `label` is added, a warning appears to inform the user that the new node will overwrite the existing one with the same label.

In [9]:
# import graph and instantiate
from uncertainty_engine.graph import Graph
graph = Graph()

In [10]:
number_node_five = Node(
    node_name="Number",
    value=5,
    label="number_node",
    client=client,
)

graph.add_node(number_node_five)

In [11]:
number_node_one = Node(
    node_name="Number",
    value=1,
    label="number_node",
    client=client,
)

graph.add_node(number_node_one)

## Validating the workflow

Constructing a workflow with problematic or incomplete inputs and passing the `client` triggers validation warnings. The workflow performs a full validation of all included nodes and their handles, even if some nodes were instantiated without a `client`. Warnings are shown for any invalid inputs that could cause the workflow to fail at runtime.

In [12]:
# create a new graph
graph_with_validation_problems = Graph()

In [13]:
# add some nodes
one_node = Node(
    node_name="Number",
    value=1,
    label="one_node",
    client=client,
)

add_node = Add(
    lhs=one_node.make_handle("value"),
    rhs=2,
    label="add_node",
    client=client,
)

graph_with_validation_problems.add_node(one_node)
graph_with_validation_problems.add_node(add_node)

In [14]:
# create a workflow from the graph
from uncertainty_engine.nodes.workflow import Workflow

workflow = Workflow(
    graph=graph_with_validation_problems.nodes,
    inputs=graph_with_validation_problems.external_input,
    external_input_id=graph_with_validation_problems.external_input_id,
    requested_output={
        "Addition result": {
            "node_name": "Add",
            "node_handle": "ans",
        },
        "Addition result alternative": add_node.make_handle("ans"),
    },
    client=client,
)


Requested Output Errors:
  - Addition result: Node with label 'Add' is referenced but is not in graph.
  - Addition result alternative: Requested output must be a dictionary, not a `Handle` object. Did you mean to use `handle.model_dump()`?
